<a href="https://colab.research.google.com/github/Iingxaki/team1_DS450-01_RH-JP/blob/Team-Mini-Project-CAPO-and-Time-to-Clinical-Stability/CAPO_and_TCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here the libraries pandas, and numpy anre imported to be able to handle the csv file and do changes to it.
Then from the library sklear.impute import SimpleImputer it is used to fill any missing categorical data.
Proceed to load the data set, filter the columns and drop the empty rows

Step 1: Data Cleaning and Preparation
Load the dataset and filter columns:

Keep only dem_, exam_, and cx_ columns.

Drop rows with excessive missing data or impute missing values where appropriate.

In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Capstone 2025/Mini Projects/capo_data.csv')

In [8]:
data.head()

,case_id,dem_age,dem_sex,dem_pregnant,dem_trimester,exam_height,exam_weight,exam_hr,exam_rr,exam_sbp,...,day6test___wbc,day6test___oral,day7test___cough,day7test___afebrile,day7test___wbc,day7test___oral,over7daytest___cough,over7daytest___afebrile,over7daytest___wbc,over7daytest___oral
0,16,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,1,1,1
1,17,60.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,18,45.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,0,0,0,0,0,0,0,0
3,19,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
4,20,61.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Filter columns
columns_to_keep = [col for col in data.columns if col.startswith(('dem_', 'exam_', 'cx_'))]
data_filtered = data[columns_to_keep].copy()  # Create an explicit copy

# Impute missing values (e.g., mean for numeric, mode for categorical)
numeric_cols = data_filtered.select_dtypes(include=np.number).columns
categorical_cols = data_filtered.select_dtypes(exclude=np.number).columns

imputer_numeric = SimpleImputer(strategy='mean')
imputer_categorical = SimpleImputer(strategy='most_frequent')

# Use .loc for imputation to avoid SettingWithCopyWarning
data_filtered.loc[:, numeric_cols] = imputer_numeric.fit_transform(data_filtered[numeric_cols])

# Check if there are any categorical columns before imputing
if len(categorical_cols) > 0:
    data_filtered.loc[:, categorical_cols] = imputer_categorical.fit_transform(data_filtered[categorical_cols])
else:
    print("Warning: No categorical columns found for imputation.")

In [10]:
data.head()

,case_id,dem_age,dem_sex,dem_pregnant,dem_trimester,exam_height,exam_weight,exam_hr,exam_rr,exam_sbp,...,day6test___wbc,day6test___oral,day7test___cough,day7test___afebrile,day7test___wbc,day7test___oral,over7daytest___cough,over7daytest___afebrile,over7daytest___wbc,over7daytest___oral
0,16,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,1,1,1
1,17,60.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,18,45.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,0,0,0,0,0,0,0,0
3,19,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
4,20,61.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Drop rows with >50% missing data
data_filtered = data_filtered.dropna(thresh=len(data_filtered.columns)//2, axis=0)

# Save cleaned dataset
data_filtered.to_csv('cleaned_cap_data.csv', index=False)

In [ ]:
def calculate_tcs(row):
    for day in range(1, 8):
        if all(row[[f'criteria1_day{day}', f'criteria2_day{day}', f'criteria3_day{day}', f'criteria4_day{day}']] == 1):
            return day
    return 8  # Right-censored if not stable by day 7

# Apply the function to calculate TCS
data_filtered['TCS'] = data_filtered.apply(calculate_tcs, axis=1)

Step 2: Survival Analysis Curve
Use the lifelines library to generate a survival curve for TCS.

In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Create a KaplanMeierFitter object
kmf = KaplanMeierFitter()

# Fit the model (TCS as event, no censoring since TCS=8 is already handled)
kmf.fit(data_filtered['TCS'], event_observed=(data_filtered['TCS'] != 8))

# Plot the survival curve
kmf.plot_survival_function()
plt.title('Survival Curve for Time to Clinical Stability (TCS)')
plt.xlabel('Days')
plt.ylabel('Percentage of Patients Clinically Stable')
plt.show()

Step 3: Linear Regression Model
Build a linear regression model to predict TCS.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Prepare features (X) and target (y)
X = data_filtered.drop(columns=['TCS'])
y = data_filtered['TCS']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2}')

Step 4: Logistic Regression Models
Binary Logistic Regression (Stable/Not Stable):

Create a binary target variable (1 if TCS ≤ 7, 0 if TCS = 8).

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Create binary target
data_filtered['Stable'] = (data_filtered['TCS'] <= 7).astype(int)

# Prepare features (X) and target (y)
X = data_filtered.drop(columns=['TCS', 'Stable'])
y = data_filtered['Stable']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

# Evaluate the model
y_pred = log_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Categorical Logistic Regression (Early/Late TCS):

Create a categorical target variable (1 if TCS < 4, 0 if TCS ≥ 4).

In [1]:
# Create categorical target
data_filtered['Early_TCS'] = (data_filtered['TCS'] < 4).astype(int)

# Prepare features (X) and target (y)
X = data_filtered.drop(columns=['TCS', 'Stable', 'Early_TCS'])
y = data_filtered['Early_TCS']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
log_model_early = LogisticRegression()
log_model_early.fit(X_train, y_train)

# Evaluate the model
y_pred = log_model_early.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

NameError: name 'data_filtered' is not defined

Step 5: Incorporate Lab Data
Add lab_ columns to the logistic regression models and evaluate performance.

In [ ]:
# Include lab data
lab_columns = [col for col in data.columns if col.startswith('lab_')]
data_with_labs = pd.concat([data_filtered, data[lab_columns]], axis=1)

# Repeat logistic regression steps with lab data included

Deliverables:
Notebook/Script:

Document each step with Markdown explanations.

Include interpretations of model performance (e.g., R2, accuracy, classification reports).

Video Presentation:

Use a narrated PowerPoint to explain the project.

Include all team members and a link to the video in the README.md.

Cleaned Dataset:

Save the cleaned dataset as cleaned_cap_data.csv.